In [2]:
import pandas as pd
import os

In [4]:

#Get current and parent directory directory
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)

#Create a path to the source folder where the raw data is located
source_path = os.path.join(parent_directory, 'data', 'raw')

#Create a path to the destination file
destination_path =os.path.join(parent_directory, 'data', 'processed', 'combine.xlsx')

# Get all .xlsx files in the source directory
file = glob.glob(os.path.join(source_path, "*.xlsx"))

#Read the first .xlsx file
df = pd.read_excel(file[0], index_col=None)

#Variable for calculating the number of changes
raw_shape = df.shape

display(df.head(10))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Grzegorz\\Documents\\GitHub\\continuity_of_absence\\data\\raw\\data_per_day.xlsx'

In [5]:
#sorting by key columns
df = df.sort_values(['Numer osobowy','Rodz. nieob./obecn.', 'Data od'])

NameError: name 'df' is not defined

In [4]:
# Define the 'combine_lines' function
def combine_lines(df):
    changes = 0
    for i in range(0, len(df)-1):
        if (i in df.index and 
            (i+1) in df.index and
            df.loc[i, 'Numer osobowy'] == df.loc[i+1, 'Numer osobowy'] and
            df.loc[i, 'Rodz. nieob./obecn.'] == df.loc[i+1, 'Rodz. nieob./obecn.'] and
            df.loc[i, 'Data do'] == df.loc[i+1, 'Data od'] - pd.Timedelta(days=1)):

            df.loc[i, 'Data do'] = df.loc[i+1, 'Data do']
            df.loc[i, 'Godziny nieobecności'] += df.loc[i+1, 'Godziny nieobecności']
            df.loc[i, 'Dni obecn./nieobecn.'] += df.loc[i+1, 'Dni obecn./nieobecn.']
            df.loc[i, 'Dni kalendarzowe'] += df.loc[i+1, 'Dni kalendarzowe']
            df.loc[i, 'Dokonano_złączenia'] = 'Tak'
            df.drop(index=[i+1], inplace=True)
            changes += 1
    
    return df, changes

In [5]:
# Define the `__main__` block
if __name__ == '__main__':
    
    #Total changes count
    total_changes = 0
    
    #Iterate until no more changes are made on df
    while True:
        new_df, changes = combine_lines(df)

        #Check if there are no more changes
        if changes == 0:
            break

        # Update the total changes count and reset the df index, does not work properly without reset 
        total_changes += changes
        df = new_df
        df.reset_index(drop=True, inplace=True)
#Print how many rows was and how many get
print(f'Combine rows - from {raw_shape[0]} to {df.shape[0]} rows.')

#Display what df looks like after the loop
display(df.head(5))

Combine rows - from 24 to 8 rows.


,Numer osobowy,Rodz. nieob./obecn.,Nieobecność,Data od,Data do,Godziny nieobecności,Dni obecn./nieobecn.,Dni kalendarzowe,Dokonano_złączenia
0,Osoba 1,NPZO,Op. Nad chor.Dzieck.,2023-03-06,2023-03-11,48,6,6,Tak
1,Osoba 1,UPZG,Płatny–MNiSzW z 12.10.06,2023-06-27,2023-07-14,112,14,16,Tak
2,Osoba 2,CHZ,Choroba zwykła,2023-03-20,2023-03-24,40,5,5,NaN
3,Osoba 2,CHZ,Choroba zwykła,2023-05-22,2023-06-02,80,10,10,Tak
4,Osoba 3,NPZO,Op. Nad chor.Dzieck.,2023-01-09,2023-12-04,248,31,32,Tak


In [6]:
#Convert columns to datetime format
df['Data od'] = pd.to_datetime(df['Data od']).dt.date
df['Data do'] = pd.to_datetime(df['Data do']).dt.date

#Save the combined df to a .xlsx file
df.to_excel(destination_path, index=False)